In [1]:
# 🔄 Auto-reload modules when their source code changes
%load_ext autoreload
%autoreload 2

# 📦 Set up src/ and utils/ paths (project root is detected via pyproject.toml)
%run ../bootstrap.py

In [2]:
import requests
import datetime
import json
from pathlib import Path
from dotenv import load_dotenv
import os
import time
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import display, clear_output

from sleeping_beauty.config.config import Config

In [3]:
load_dotenv()

True

In [4]:
config = Config()


PROJECT_ROOT = Path(config.PROJECT_ROOT)
config.config_path = str(PROJECT_ROOT / "configs" / "config.yaml")
# "configs/FE/features_config.yaml"
config.load_from_yaml(config.config_path)




[Config] Loaded YAML config: /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml


In [5]:
config.print_config_info()

📂 Configuration
--------------------------------------------------
Configuration file:       /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml
--------------------------------------------------
🔐 Auth / Oura
--------------------------------------------------
Token path:               /Users/kenneth/.sleeping_beauty/oura_token.json
Client ID set:            True
Client Secret set:        True
Redirect URI:             http://localhost:8400/callback
Scopes:                   ['daily', 'personal']


In [6]:
from datetime import date, timedelta
import asyncio

from sleeping_beauty.clients.oura_api_client import OuraApiClient
from sleeping_beauty.oura.auth.oura_auth import OuraAuth  
from sleeping_beauty.oura.auth.storage.file_storage import FileTokenStorage

In [7]:
storage = FileTokenStorage(
    path=Path(
        config.oura_token_path
    ).expanduser()
)

# Instantiate your existing auth service
oura_auth = OuraAuth.from_config()
oura_auth.preflight_check()

# Token provider callable (what the client expects)
token_provider = oura_auth.get_access_token   ## notice this is passing the get_access_token as the callable function 

# Create the API client
client = OuraApiClient(token_provider=token_provider)

🔐 Oura Auth Preflight Check
----------------------------------------
✅ client_id present
✅ client_secret present
✅ redirect_uri: http://localhost:8400/callback
✅ requested scopes: ['extapi:daily', 'extapi:personal']
✅ token found in storage
✅ token valid for ~43158 minutes
✅ token covers requested scopes
----------------------------------------
✔️  Preflight check complete


In [8]:
async def call_personal_info():
    info = await client.get_personal_info()
    return info


personal_info = await call_personal_info()
personal_info

PersonalInfo(user_id='7-8b292ba-5951--7-5949b073a592d6c-81', age=59, biological_sex='male', height=1.6, weight=67.3, email=None, raw={'id': '7-8b292ba-5951--7-5949b073a592d6c-81', 'age': 59, 'weight': 67.3, 'height': 1.6, 'biological_sex': 'male', 'email': None})

In [9]:
end = date.today() - timedelta(days=1)
start = end - timedelta(days=1)

page = await client.get_daily_sleep_score_page(
    start_date=start,
    end_date=end,
)

page.raw.keys(), page.raw

(dict_keys(['data', 'next_token']),
 {'data': [{'id': '3a1ad9ad-c36e-44ee-b622-45da6e67a3b6',
    'contributors': {'deep_sleep': 78,
     'efficiency': 95,
     'latency': 97,
     'rem_sleep': 96,
     'restfulness': 63,
     'timing': 100,
     'total_sleep': 75},
    'day': '2026-01-09',
    'score': 82,
    'timestamp': '2026-01-09T00:00:00.000+00:00'},
   {'id': 'a06824c2-8028-4f27-844b-113ce981dab7',
    'contributors': {'deep_sleep': 96,
     'efficiency': 96,
     'latency': 83,
     'rem_sleep': 94,
     'restfulness': 69,
     'timing': 100,
     'total_sleep': 75},
    'day': '2026-01-10',
    'score': 84,
    'timestamp': '2026-01-10T00:00:00.000+00:00'}],
  'next_token': None})

In [11]:
item = page.raw["data"][0]
item.keys(), item

(dict_keys(['id', 'contributors', 'day', 'score', 'timestamp']),
 {'id': '3a1ad9ad-c36e-44ee-b622-45da6e67a3b6',
  'contributors': {'deep_sleep': 78,
   'efficiency': 95,
   'latency': 97,
   'rem_sleep': 96,
   'restfulness': 63,
   'timing': 100,
   'total_sleep': 75},
  'day': '2026-01-09',
  'score': 82,
  'timestamp': '2026-01-09T00:00:00.000+00:00'})

In [14]:
doc_id = page.data[0].id

single = await client.get_daily_sleep_score(document_id=doc_id)
single



DailySleepScore(id='3a1ad9ad-c36e-44ee-b622-45da6e67a3b6', day=datetime.date(2026, 1, 9), score=82, deep_sleep=78, efficiency=95, latency=97, rem_sleep=96, restfulness=63, timing=100, total_sleep=75, timestamp='2026-01-09T00:00:00.000+00:00', raw={'id': '3a1ad9ad-c36e-44ee-b622-45da6e67a3b6', 'contributors': {'deep_sleep': 78, 'efficiency': 95, 'latency': 97, 'rem_sleep': 96, 'restfulness': 63, 'timing': 100, 'total_sleep': 75}, 'day': '2026-01-09', 'score': 82, 'timestamp': '2026-01-09T00:00:00.000+00:00'})